In [1]:
!wget 'https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKtc-hinted.zip'
!mkdir /tmp/fonts
!unzip -o NotoSansCJKtc-hinted.zip -d /tmp/fonts/
!mv /tmp/fonts/NotoSansMonoCJKtc-Regular.otf /usr/share/fonts/truetype/NotoSansMonoCJKtc-Regular.otf -f
!rm -rf /tmp/fonts
!rm NotoSansCJKtc-hinted.zip
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt

font_dirs = ['/usr/share/fonts/truetype/']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
  font_manager.fontManager.addfont(font_file)

plt.rcParams['font.family'] = "Noto Sans Mono CJK TC"

--2022-03-22 03:18:10--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKtc-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 108.177.125.128, 2404:6800:4008:c01::80
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121247366 (116M) [application/zip]
Saving to: ‘NotoSansCJKtc-hinted.zip’

NotoSansCJKtc-hinte 100%[===================>] 115.63M   154MB/s    in 0.7s    

2022-03-22 03:18:11 (154 MB/s) - ‘NotoSansCJKtc-hinted.zip’ saved [121247366/121247366]

Archive:  NotoSansCJKtc-hinted.zip
  inflating: /tmp/fonts/LICENSE_OFL.txt  
  inflating: /tmp/fonts/NotoSansCJKtc-Black.otf  
  inflating: /tmp/fonts/NotoSansCJKtc-Bold.otf  
  inflating: /tmp/fonts/NotoSansCJKtc-DemiLight.otf  
  inflating: /tmp/fonts/NotoSansCJKtc-Light.otf  
  inflating: /tmp/fonts/NotoSansCJKtc-Medium.otf  
  inflating

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import tejapi

In [9]:
pip install tejapi

In [10]:
tejapi.ApiConfig.api_key= ###yourkey
tejapi.ApiConfig.info()
tejapi.ApiConfig.ignoretz = True

In [11]:

data=tejapi.get('TWN/EWNPRCSTD' ,chinese_column_name=True )
select=data["上市別"].unique()
select=select[1:3]
condition =(data["上市別"].isin(select)) & ( data["證券種類名稱"]=="普通股" )
data=data[condition]
twid=data["證券碼"].to_list()

In [80]:
year='2017'
start= year+'-01-01'
end  = year+"-12-31"
dividend=tejapi.get('TWN/AREINT' ,
                  coid=twid,
            mdate={'gt':start,'lt':end},
            paginate=True,
            chinese_column_name=True)
dividend.set_index("證券碼",inplace=True)

In [14]:
dividend.head()

,證券碼,除權息日,股東會日期,分配年度,市場別,除權息類別,現金股息_每股元,股票股利_每股元,除權息前收盤價,除息前指數收盤價,...,填權息日,填權息日最高價,填權息日收盤價,填權息天數,除權息日前一週報酬率,除權息日後一週報酬率,現金股利發放日,股票股利發放日,填權息日_收盤價註記,填權息(Y/N)
None,,,,,,,,,,,,,,,,,,,,,
0,1101,2017-07-25,2017-06-28,2016-01-01,TSE,除息,1.45,0.0,36.00,10461.28,...,2017-12-25,36.25,36.15,108,2.1337,1.3025,2017-08-25,NaT,HC,Y
1,1102,2017-08-30,2017-06-27,2016-01-01,TSE,除息,0.90,0.0,27.40,10496.57,...,2017-09-01,27.60,27.45,3,0.9258,3.5344,2017-09-27,NaT,HC,Y
2,1103,2017-08-24,2017-06-19,2016-01-01,TSE,除息,0.20,0.0,10.90,10406.81,...,2017-08-24,11.00,10.95,1,-1.4019,12.1026,2017-09-15,NaT,HC,Y
3,1104,2017-07-14,2017-06-14,2016-01-01,TSE,除權除息,1.20,0.3,25.15,10460.15,...,NaT,NaN,NaN,0,-0.7969,-1.0851,2017-08-18,2017-08-18,,N
4,1108,2017-07-26,2017-06-07,2016-01-01,TSE,除息,0.60,0.0,9.54,10463.15,...,NaT,NaN,NaN,0,0.9574,-0.6219,2017-08-15,NaT,,N


In [17]:
start= year+'-05-09'
end=   year+"-05-11"
opts={'columns': ['coid','mdate', 'beta_1y']}
beta = tejapi.get('TWN/ABETA',
                  coid=twid,
            mdate={'gt':start,'lt':end},
             opts=opts,
            paginate=True,
            chinese_column_name=True
)
beta.set_index("證券代碼",inplace=True) #方便合併

In [18]:
beta.head()

,日期,CAPM_Beta 一年
證券代碼,,
1101,2017-05-10,1.1461
1102,2017-05-10,1.1382
1103,2017-05-10,0.5984
1104,2017-05-10,0.5603
1108,2017-05-10,0.2470


In [19]:
start= year+'-05-9'
end=   year+"-05-11"
getperatio=tejapi.get('TWN/APRCD' ,
                  coid=twid,
                  mdate={'gt':start,'lt':end},
                  paginate=True,
                  chinese_column_name=True )
getperatio.set_index("證券代碼",inplace=True)

In [21]:
getperatio.head()

,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),報酬率％,週轉率％,流通在外股數(千股),...,現金股利率,股價漲跌(元),高低價差%,次日開盤參考價,次日漲停價,次日跌停價,注意股票(A),處置股票(D),全額交割(Y),市場別
證券代碼,,,,,,,,,,,,,,,,,,,,,
1101,2017-05-10,34.65,34.75,34.50,34.75,3403.788,117819.026,0.2886,0.0922,3692176,...,4.17,0.10,0.7215,34.75,38.20,31.30,,,,TSE
1102,2017-05-10,28.70,29.15,28.70,28.90,1689.301,48835.807,0.6969,0.0502,3361447,...,3.11,0.20,1.5679,28.90,31.75,26.05,,,,TSE
1103,2017-05-10,10.35,10.50,10.30,10.35,325.728,3381.795,0.0000,0.0419,774781,...,0.97,0.00,1.9324,10.35,11.35,9.32,,,,TSE
1104,2017-05-10,25.80,26.00,25.80,25.85,71.168,1841.327,-0.1931,0.0112,634572,...,4.64,-0.05,0.7722,25.85,28.40,23.30,,,,TSE
1108,2017-05-10,9.55,9.63,9.55,9.63,162.277,1552.246,0.6270,0.0400,404738,...,6.23,0.06,0.8359,9.63,10.55,8.67,,,,TSE


In [24]:
twid1=twid 
groups = []
while True:
    if len(twid1) >= 50:
        groups.append(twid1[:50])
        twid1 = twid1[50:]
    elif 0 <= len(twid1) < 50:
        groups.append(twid1)
        break


In [26]:

fin_data = pd.DataFrame()

start=year+'-01-01'
end=  year+"-03-22"
for group in groups:
    fin_data = fin_data.append(tejapi.get('TWN/EWIFINQ',
                  coid = group,
                  mdate={'gt':start,'lt':end},
                 
                  chinese_column_name = True,
                  paginate = True)).reset_index(drop=True)


fin_data.set_index("證券碼",inplace=True)

In [27]:
fin_data.head()

,財務資料日,季別,合併(Y/N),單季(Q)/單半年(H),月份,幣別,財報發布日,ROA(C) 稅前息前折舊前,ROE(A)-稅後,營業毛利率,...,歸屬母公司淨利（損）,每股盈餘,普通股每股現金股利（盈餘及公積）,普通股每股股票股利－盈餘,普通股每股股票股利－公積,折舊－CFO,攤提－CFO,來自營運之現金流量,投資活動之現金流量,籌資活動之現金流量
證券碼,,,,,,,,,,,,,,,,,,,,,
1101,2017-03-01,1,Y,Q,03,NTD,2017-05-15,5.92,3.16,14.16,...,832244.0,0.92,None,None,None,1530197.0,96971.0,1408140.0,306706.0,-3768585.0
1102,2017-03-01,1,Y,Q,03,NTD,2017-05-15,3.04,0.40,7.19,...,236681.0,0.32,None,None,None,1218619.0,50727.0,-282365.0,-5983834.0,8780653.0
1103,2017-03-01,1,Y,Q,03,NTD,2017-05-15,0.32,-2.64,22.15,...,-111320.0,-0.68,None,None,None,91393.0,0.0,411500.0,-426352.0,-889892.0
1104,2017-03-01,1,Y,Q,03,NTD,2017-05-15,6.20,6.80,12.63,...,283869.0,1.80,None,None,None,24219.0,433.0,15565.0,-111139.0,201117.0
1108,2017-03-01,1,Y,Q,03,NTD,2017-05-15,5.28,1.56,9.03,...,18766.0,0.20,None,None,None,68523.0,2464.0,-10179.0,56463.0,-79592.0


In [28]:
opts={'columns': ['coid','mdate','d0007']}

start=year+'-04-30'
end=year+"-05-15"
trade=tejapi.get('TWN/ASALE' ,
            coid=twid,
            mdate={'gt':start,'lt':end},
            opts=opts,
            paginate=True,
            chinese_column_name=True)   
trade.set_index("公司",inplace=True)

In [29]:
trade.head()

,年月,累計營收成長率%
公司,,
1101,2017-05-01,15.77
1102,2017-05-01,-1.74
1103,2017-05-01,-15.59
1104,2017-05-01,-5.83
1108,2017-05-01,-13.99


In [81]:
dividend["累計營收成長率%"]=trade["累計營收成長率%"]
dividend["股利殖利率-TSE"]=getperatio["股利殖利率-TSE" ]
dividend["現金股利率"]=getperatio["現金股利率" ]
dividend["營業毛利率"]=fin_data["營業毛利率"]
dividend["CAPM_Beta 一年"]=beta['CAPM_Beta 一年']
dividend["填權息(Y/N)"]=dividend["填權息(Y/N)"].apply(lambda x : 1 if x == "Y" else 0)
dividend["填權息(Y/N)"].value_counts() 
dividend["本益比"]=getperatio["本益比-TSE" ]
dividend["營業毛利成長率"]=fin_data["營業利益成長率"]


In [50]:
dividend.head()

,除權息日,股東會日期,分配年度,市場別,除權息類別,現金股息_每股元,股票股利_每股元,除權息前收盤價,除息前指數收盤價,除息前市值比重,...,現金股利發放日,股票股利發放日,填權息日_收盤價註記,填權息(Y/N),累計營收成長率%,股利殖利率-TSE,現金股利率,營業毛利率,CAPM_Beta 一年,本益比
證券碼,,,,,,,,,,,,,,,,,,,,,
1101,2017-07-25,2017-06-28,2016-01-01,TSE,除息,1.45,0.0,36.00,10461.28,0.429,...,2017-08-25,NaT,HC,1,15.77,4.17,4.17,14.16,1.1461,20.20
1102,2017-08-30,2017-06-27,2016-01-01,TSE,除息,0.90,0.0,27.40,10496.57,0.297,...,2017-09-27,NaT,HC,1,-1.74,3.11,3.11,7.19,1.1382,24.70
1103,2017-08-24,2017-06-19,2016-01-01,TSE,除息,0.20,0.0,10.90,10406.81,0.027,...,2017-09-15,NaT,HC,1,-15.59,0.97,0.97,22.15,0.5984,258.75
1104,2017-07-14,2017-06-14,2016-01-01,TSE,除權除息,1.20,0.3,25.15,10460.15,0.051,...,2017-08-18,2017-08-18,,0,-5.83,5.80,4.64,12.63,0.5603,9.65
1108,2017-07-26,2017-06-07,2016-01-01,TSE,除息,0.60,0.0,9.54,10463.15,0.012,...,2017-08-15,NaT,,0,-13.99,6.23,6.23,9.03,0.2470,12.35


In [65]:
def count(x):
    a,b =  x["填權息(Y/N)"].value_counts() 
    c=x["填權息天數"].mean()
    return f'填息機率 : {a/(a+b)}' , f'填息家數 : {a}' ,f'未填息家數 : {b}', f'填息天數 : {c}'
    

In [82]:
div5=dividend[dividend["股利殖利率-TSE"]>5]



In [83]:
count(div5)

('填息機率 : 0.7810526315789473',
 '填息家數 : 371',
 '未填息家數 : 104',
 '填息天數 : 46.14947368421053')

In [84]:
def capmselect(df,b,c):
    yy=df[(df["股利殖利率-TSE"]>5)&(df["CAPM_Beta 一年"]>b)&(df["CAPM_Beta 一年"] < c)]
    return count(yy)
def capmpeselect(df,b,c,d ): ########條件增加月營收的
    yy=df[(df["股利殖利率-TSE"]>5)&(df["CAPM_Beta 一年"]>b)&(df["CAPM_Beta 一年"] < c)& (df["累計營收成長率%"] > d )]
    return count(yy)
def ncapmpeselect(df,b,c,d ):#####累計成長負的
    yy=df[(df["股利殖利率-TSE"]>5)&(df["CAPM_Beta 一年"]>b)&(df["CAPM_Beta 一年"] < c)& (df["累計營收成長率%"] < d )]
    return count(yy)
def allcondition(a,c,d,e,f): #累計營收正 毛利率成長正
    allcondition=dividend[(dividend["股利殖利率-TSE"]>a)&(dividend["CAPM_Beta 一年"]>c)&(dividend["CAPM_Beta 一年"]<d)&(dividend["累計營收成長率%"]>e)&(dividend["營業毛利成長率"]>f)]
    return count(allcondition)
def nallcondition(a ,c,d,e,f): #累計營收負 毛利率成長負
    allcondition=dividend[(dividend["股利殖利率-TSE"]>a)&(dividend["CAPM_Beta 一年"]>c)&(dividend["CAPM_Beta 一年"]<d)&(dividend["累計營收成長率%"]< e)&(dividend["營業毛利成長率"]<f)]
    return count(allcondition)

In [87]:
capmselect(dividend,0.4,0.6) #capm係數 0.4 ~ 0.6


('填息機率 : 0.7476635514018691',
 '填息家數 : 80',
 '未填息家數 : 27',
 '填息天數 : 47.5607476635514')

In [88]:
capmpeselect(dividend,0.4,0.6,0) #capm係數 0.4 ~ 0.6 累計月營收正


('填息機率 : 0.8431372549019608',
 '填息家數 : 43',
 '未填息家數 : 8',
 '填息天數 : 44.84313725490196')

In [89]:
ncapmpeselect(dividend,0.4,0.6,0) #capm係數 0.4 ~ 0.6 累計月營收負


('填息機率 : 0.6607142857142857',
 '填息家數 : 37',
 '未填息家數 : 19',
 '填息天數 : 50.035714285714285')

In [90]:
allcondition(5,0.4,0.6,0,0) #capm係數 0.4 ~ 0.6 累計月營收毛利率正成長


('填息機率 : 0.9411764705882353',
 '填息家數 : 32',
 '未填息家數 : 2',
 '填息天數 : 45.088235294117645')

In [91]:
nallcondition(5,0.4,0.6,0,0) #capm係數 0.4 ~ 0.6 累計月營收毛利率負成長


('填息機率 : 0.65625', '填息家數 : 21', '未填息家數 : 11', '填息天數 : 54.125')